NOMS I COGNOMS:

GRUP:

# Naive Bayes i Classificació


En aquest quart lliurament es programarà un classificador, que donat un tweet el categoritzarà en una de les possibles classes. En aquesta ocasió, implementareu un classificador amb tweets de cyber bullying.


**Què s’ha de fer?**

Volem classificar tweets segons a quin tipus de cyber bullying pertanyen. Així doncs, a partir de tots els tweets que tenim, crearem un vector de característiques que ens descrigui cadascun. Finalment desenvoluparem un classificador probabilístic del tipus Naive Bayes que ens permeti identificar a quina classe de cyber bullying pertany un tweet donat segons les característiques triades.


**Quina és la idea del sistema de classificació que s’ha de desenvolupar?**

El classificador és un concepte de l'aprenentatge automàtic supervisat. L'objectiu del classificador és donat un vector de característiques que descriuen els objectes que es volen classificar indicar a quina categoria o classe pertanyen d'entre un conjunt predeterminat. 

El procés de classificació consta de dues parts: 
(a) el procés d'aprenentatge i 
(b) el procés d'explotació o testeig. 
El procés d'aprenentatge rep exemples de parelles $(x,y)$ on $x$ són les característiques, usualment nombres reals, i $y$ és la categoria a la que pertanyen. 
Aquest conjunt se'l coneix com a conjunt d'entrenament i ens servirà per trobar una funció $\hat{y}=h(x)$ que donada una $x$ aconsegueixi que $\hat{y}=y$. Per altra banda el procés de testeig aplica la funció $h(x)$ apresa a l'entrenament a una nova descripció per veure quina categoria li correspon.


**Classificació i llenguatge natural**

La descripció dels exemples en característiques és el punt més crític de tot sistema d'aprenentatge automàtic. 
Una de les representacions més simples per tal de descriure un text és la representació *bag-of-words*.
Aquesta representació converteix un text en un vector de $N$ paraules. 
Consisteix en seleccionar un conjunt d'$N$ paraules i per cada paraula comptar quants cops apareix en el text. 
Una versió alternativa d'aquest procés pot ser simplement indicar si apareix o no en el text.

## Abans de començar


**\+ Durant la pràctica, solament es podran fer servir les següents llibreries**:

`Pandas, Numpy` i `NLTK`

*Nota: A més de les que ja es troben presents en la 1a cel·la i funcions natives de Python*

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica que serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple (ho posarà en el pydoc de la funció), `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades. Durant els testos, els paràmetres (i específicament `df`) no contindran les mateixes dades que en aquest notebook, si bé si seran del mateix tipus! Per tant, no us refieu de què tinguin, per exemple, el mateix nombre de files.

## Més informació del dataset

El 15 d'Abril de 2020, UNICEF va llançar una alarma com a resposta de l'augment de risc de cyberbullying durant la pandèmia COVID-19. Les estadístiques són prou alarmants: un 36.5% dels estudiants de l'escola fins a l'institut s'han sentit víctimes del cyberbullying i un 87% l'han observat, amb efectes que van des d'una disminució de resultats acadèmics fins a pensaments suïcides.

Amb l'objectiu d'ajudar a l'analisis de la situació, s'ha construit un dataset que conté més de 47000 tweets etiquetats d'acord amb la classe de cyberbullying que s'està donant:

1. Age;
2. Ethnicity;
3. Gender;
4. Religion;
5. Other type of cyberbullying;
6. Not cyberbullying

Les dades han estat balancejades per tal de contenir aproximadament 8000 mostres de cada classe.

# Preparar les dades

## Llegim dades

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('data/cyberbullying_tweets.csv')
df

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depe...",ethnicity
47688,Turner did not withhold his disappointment. Tu...,ethnicity
47689,I swear to God. This dumb nigger bitch. I have...,ethnicity
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,ethnicity


In [5]:
df['cyberbullying_type'].value_counts()

religion               7998
age                    7992
gender                 7973
ethnicity              7961
not_cyberbullying      7945
other_cyberbullying    7823
Name: cyberbullying_type, dtype: int64

## Dividim dataset

Dividim els tweets en un conjunt d'entrenament, *train*, i en un conjunt de validació, *test*, per tal de poder entrenar i validar el nostre model de ML.

In [6]:
from sklearn.model_selection import train_test_split

df_tweets_train, df_tweets_test = train_test_split(df, test_size=0.2)

Com les dades estaven balancejades originalment, podem observar que la distribució de cadascuna de les classes es manté:

In [7]:
df_tweets_train['cyberbullying_type'].value_counts()

religion               6437
ethnicity              6407
not_cyberbullying      6396
gender                 6347
age                    6338
other_cyberbullying    6228
Name: cyberbullying_type, dtype: int64

In [8]:
df_tweets_test['cyberbullying_type'].value_counts()

age                    1654
gender                 1626
other_cyberbullying    1595
religion               1561
ethnicity              1554
not_cyberbullying      1549
Name: cyberbullying_type, dtype: int64

# Implementació

Dividirem el notebook en 3 seccions que es complementen una a l'altra:

1. Anàlisis de dades: Informació bàsica sobre els tweets
2. Processament de les dades: Creació d'un vector de característiques a partir dels tweets
3. Classificació amb Naive Bayes

### 1. Anàlisis de dades

El primer que haurem de fer és analitzar les dades per veure una mica com són. El que us proposem és fer una sèrie de plots per observar dades com ara:

* quants tweets s'estan dirigint a una persona en concret
* quants hastags hi ha a cada categoria de tweets
* quants tweets hi ha de cada categoria
* quants tweets de la categoria "not_cyberbullying" és dirigeixen a un usuari vs totes les altres categories
* altres coses que penseu que poden ser rellevants

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
df_tweets_train.head()

,tweet_text,cyberbullying_type
7339,@ScottBass because my sunday school had a part...,not_cyberbullying
8295,@thehiredmind You can have this one. Untag. T...,gender
1469,@WyattJamez good stuff👍,not_cyberbullying
2507,@SexKittenParty White feminists need to focus ...,not_cyberbullying
45255,RT @SpeakingOfKe_: If a nigga ever talk bad ab...,ethnicity


#### **EXERCICI 1:** FEU EL VOSTRE ANALISIS DE DADES AQUÍ

In [11]:
def analisis(df):
    return

analisis(df_tweets_train)

### Comptar paraules

El primer que haurem d'implementar és la funció *normalize* que normalitzarà les paraules.


No modificar la següent cel·la, s'encarrega de fer el proce

In [12]:
def memo(f):
    class memodict(dict):
        def __init__(self, f):
            self.f = f
        def __call__(self, *args):
            return self[args]
        def __missing__(self, key):
            ret = self[key] = self.f(*key)
            return ret
    return memodict(f)

#### **EXERCICI 2:** 

Empleneu la funció següent que, donada una paraula, la normalitzi passant tots els digits a minúscules.

In [13]:
@memo    
def normalize(word):
    """
    Funció que donada una paraula la normalitzi
    Exemple: Taller DELS noUS USOS ---> tallers dels nous usos
    
    :param word: paraula a normalitzar
    :return : paraula normalitzada
    """
    # mantenim només les lletres i espais
    word = ''.join([i for i in word if i.isalpha() or i.isspace()])
    # borrem espais dels dos costats i ho passem a minuscula
    word = word.lower().strip()
   
    return word

In [14]:
normalize(' Taller DELS noUS USOS 1###22')

'taller dels nous usos'

#### **EXERCICI 3:** 

Feu una funció que construeixi un diccionari que contingui totes les paraules que s'han trobat tot indicant el total de cops que ha aparegut cadascuna i el nombre de tweets on apareix. Més a baix teniu un exemple de l'estructura que ha de tenir el output de la funció.

In [15]:
wet = '@GuysPIctures fuck Lincoln dumb nigger. Lover'

for word in wet.split():
    print(word)

@GuysPIctures
fuck
Lincoln
dumb
nigger.
Lover


In [16]:
def count_words(df):
    """
    Funció que ha de construir un diccionari que contingui totes les paraules que s'han trobat indicant
    el total de cops que ha aparegut i el nombre de tweets on apareix
    
    :param df: DataFrame amb els tweets i la informació associada
    :return : Diccionari amb el format {word : {n_ocur: valor, n_tweets: valor}, ...}
    """
    word_dicc = {}
    visited = set()
    
    for tweet in df['tweet_text']:
        seen_in_tweet = set()
        for word in tweet.split():
            word = normalize(word)
            if word not in visited: 
                visited.add(word)
                # creem un diccionari per la paraula
                word_dicc[word] = {}
                word_dicc[word]['n_ocur'] = 0
                word_dicc[word]['n_tweets'] = 0
            if word not in seen_in_tweet:
                word_dicc[word]['n_tweets'] += 1
                seen_in_tweet.add(word)
            word_dicc[word]['n_ocur'] += 1
    return word_dicc
dicc_text = count_words(df_tweets_train)
print(len(dicc_text))

55986


In [17]:
dicc_text = count_words(df_tweets_train)
print (len(dicc_text))

55986


El resultat serà un diccionari tipus (no necessàriament amb aquest valors):

```python
{
    'memory' : {'n_ocur': 88, 'n_tweets': 76},
    'best': {'n_ocur': 123, 'n_tweets': 65},
    ...
}
```

### Contar paraules per cada categoria de tweet

In [18]:
df_tweets_train.head()

,tweet_text,cyberbullying_type
7339,@ScottBass because my sunday school had a part...,not_cyberbullying
8295,@thehiredmind You can have this one. Untag. T...,gender
1469,@WyattJamez good stuff👍,not_cyberbullying
2507,@SexKittenParty White feminists need to focus ...,not_cyberbullying
45255,RT @SpeakingOfKe_: If a nigga ever talk bad ab...,ethnicity


#### **EXERCICI 4:** 

Fent servir la funció que se us dona a continuació (eachTopic), apliqueu-la per tal de comptar les paraules que s'han trobat i la seva ocurrència segregant ara per categoria.

In [23]:
#group = df.groupby(['cyberbullying_type'])
#group.get_group('religion')

In [24]:
"""words_topic = {}

oops = df['cyberbullying_type'].iloc[0]
#print(oops)

oo = df['cyberbullying_type'].value_counts()
ind = oo.index.values
print(ind)
print(type(ind))

for i in ind:
    print(i)
    group = df.groupby(['cyberbullying_type'])
    group = group.get_group(i)
    print(group)
"""

"words_topic = {}\n\noops = df['cyberbullying_type'].iloc[0]\n#print(oops)\n\noo = df['cyberbullying_type'].value_counts()\nind = oo.index.values\nprint(ind)\nprint(type(ind))\n\nfor i in ind:\n    print(i)\n    group = df.groupby(['cyberbullying_type'])\n    group = group.get_group(i)\n    print(group)\n"

In [25]:
def count_words_categories(df):
    """
    Funció que ha de constuir un diccionari que conté la freqüència de les 
    paraules i el número de tweets on ha aparegut. 
    Aquesta informació ha de ser dividida per diferents categories de cyberbullying.
    
    :param df: DataFrame amb els tweets i la informació associada
    :return : Diccionari amb el format {label : {word : {n_ocur: valor, n_news: valor} } }
    """
    words_topic = {}
    
    def eachTopic(group):
        # Count words on this topic and save to dictionary
        words_topic[group['cyberbullying_type'].iloc[0]] = count_words(group)

    # Group by topics and apply function to each topic
    
    # obtenim els indexos en un array per poder obtenir el nom de cada tipus de cyberbulling
    cyberbullings = df['cyberbullying_type'].value_counts().index.values
    for cyberbulling in cyberbullings:
        group = df.groupby(['cyberbullying_type'])
        group = group.get_group(cyberbulling)
        eachTopic(group)
    
    return words_topic

words_categories = count_words_categories(df_tweets_train)


In [26]:
words_categories = count_words_categories(df_tweets_train)
print (len(words_categories))

6


El resultat serà un diccionari tipus (no necessàriament amb aquest valors):

```python
{
    'ethnicity': {
        'race' : {'n_ocur': 88, 'n_tweets': 76},
        'what': {'n_ocur': 123, 'n_tweets': 65}
        ...
    },
    ...
    'gender': {
        'jokes' : {'n_ocur': 18, 'n_tweets': 17},
        'you': {'n_ocur': 154, 'n_tweets': 66}
    }
    ...
}
```

### Paraules més freqüents als tweets


**El problema de com escollir el vector de carecterístiques**

L'elecció de les paraules que formen el vector de característiques és un pas crític. En funció de com de bona sigui aquesta descripció, millor funcionarà el sistema. Tot i que us deixem a vosaltres la política de creació del vector de característiques us donem una pista: per saber quines paraules fer servir una possible estratègia és agafar aquelles paraules que apareixen entre en un 10 i un 50 percent del total (sense tenir en compte la categoria). 

Podeu experimentar variant aquests valors.

#### **EXERCICI 5:** 

Experimenteu omplint la llista *skip_top* amb aquelles paraules que penseu no tenen significat o relevancia per definir cada categoria. Podeu buscar informació sobre **stop words** a internet i definir varies llistes fins que penseu que obteniu una bona representació de paraules per categoria de cyberbullying.

In [27]:
skip_top = ("hi", "bye","do","and","to","from","that","i","you","be","her",
           "some","of","this","would","a","have","make","which","like","as","but",
           "by","but","with","on","we","say","they","his","my","an","there","what",
           "up","out","who","get","if","about","when","can","them","also","well",
           "want","because","our","even","most","us","any","way","the","in","me","was","",
           "for","are","at","all","she","he","is","it","so","just","not","being","were","its",
           "got","one","people","im","now","had","middle","how","your","their","done","no",
           "know","him","never","u","your","dont","or","then","these","into","really","why",
            "been","first","think","didnt","other","will","did","still","has","mean","go",
            "years","life","only","always","went","to","every","too","over","more","used","those",
           "ur","rt","cant","see","going","said","getting","called","past","need","thats",
           "should","time")

def topNwords(df, words, N, skip=[]):
    """
    Funció que crea un diccionari amb les N paraules més representatives 
    (les que apareixen amb més freqüència) de cadascuna de les categories de cyberbullying.
    
    Tingueu en compte que també haureu de filtrar aquelles paraules que apareixen en la majoria 
    de tweets, així com també, les que únicament apareixen en un conjunt molt petit de tweets
    
    :param df: DataFrame amb els tweets i la informació associada
    :param words: diccionari amb les paraules i la seva frequencia
    :param N: número de paraules més representatives que volem considerar
    :return : Diccionari amb el format {categoria1: llista_top_words_cat_1,  
                                        categoria2: llista_top_words_cat_2, ...} 
    """
    top_words=dict()
    
    def each_word(topic, word):
        if word not in skip:
            return words[topic][word]['n_ocur']
        return 0
    
    for topic in words:
        top_words[topic] = sorted(words[topic], key=lambda x: each_word(topic, x))[-N:][::-1]
    
    return top_words

In [28]:
top_words = topNwords(df_tweets_train, words_categories, 20, skip_top)
#top_words

El resultat serà un diccionari tipus (no necessàriament amb aquest valors):

```python
{
    'age': ['school', 'high', ...],
    ...
    'religion': ['muslims', 'christian',...]
    ...
}
```

> Una pista de que aneu ben encaminats es que per cadascuna de les categories de cyberbullying obtingueu paraules rellevants per aquesta. Si no es així, vol dir que heu d'incrementar el nombre de paraules a saltar (*skip_top*).

> EXPERIMENTEU AQUÍ QUÈ PASSA SEGONS LES "STOP WORDS" QUE USEU.

### Vector de Característiques

#### **EXERCICI 6:** 

Creeu el vector de característiques necessari per a fer l’entrenament del Naïve Bayes.

In [53]:
"""for index,tweet in enumerate(df['tweet_text']):
    print(tweet)
    print(index)
    break"""

"for index,tweet in enumerate(df['tweet_text']):\n    print(tweet)\n    print(index)\n    break"

In [52]:
"""words = []
for frequent_words in pd.Series(top_words).values:
    for word in frequent_words:
        if word not in words:
            words.append(word)
words = np.array(words)
#print(words)

def tweet_normalized(tweet):
    words = set()
    for word in tweet.split():
        words.add(normalize(word))
    return words

a_tweet = df['tweet_text'][0]
print(a_tweet)

norm = tweet_normalized(a_tweet)
print(norm)

llista = np.zeros((len(words),), dtype=int)
print(llista)
for index,word in enumerate(words):
    if word in norm:
        llista[index] = 1
        
print(llista)"""

"words = []\nfor frequent_words in pd.Series(top_words).values:\n    for word in frequent_words:\n        if word not in words:\n            words.append(word)\nwords = np.array(words)\n#print(words)\n\ndef tweet_normalized(tweet):\n    words = set()\n    for word in tweet.split():\n        words.add(normalize(word))\n    return words\n\na_tweet = df['tweet_text'][0]\nprint(a_tweet)\n\nnorm = tweet_normalized(a_tweet)\nprint(norm)\n\nllista = np.zeros((len(words),), dtype=int)\nprint(llista)\nfor index,word in enumerate(words):\n    if word in norm:\n        llista[index] = 1\n        \nprint(llista)"

In [48]:
def create_features(df, top_words): 
    """
    Funció que crea un vector de característiques necessari per a l'entrenament del classificador Naive Bayes
    
    :params df: DataFrame amb els tweets i la informació associada
    :params top_words: ha de ser el diccionari que retorna topNWords
    :return : diccionari o pd.Series que conté un np.array per a 
        cadascuna dels tweets amb el vector de característiques corresponent.
    """
    
    dict_feat_vector = {}
    
    # creem el vector de característiques amb totes les paraules freqüents
    words = []
    for frequent_words in pd.Series(top_words).values:
        for word in frequent_words:
            if word not in words:
                words.append(word)
    words = np.array(words)
    
    def tweet_normalized(tweet):
        words = set()
        for word in tweet.split():
            words.add(normalize(word))
        return words
    
    # analitzem cada tweet i mirem si cada paraula del tweet esta en la nostra llista de paraules freqüents
    for index,tweet in enumerate(df['tweet_text']):
        for i,word in enumerate(words):
            # inicialitzem
            if i == 0:
                dict_feat_vector[index] = np.zeros((len(words),), dtype=int)
            if word in tweet_normalized(tweet):
                dict_feat_vector[index][i] = 1
       
    return dict_feat_vector


In [49]:
N = 20 # Aquest parametre el podem canviar i fer proves per avaluar quin és el millor valor. 
words_categories = count_words_categories(df_tweets_train)
top_words = topNwords(df_tweets_train, words_categories, N, skip_top)
dict_feat_vector = create_features(df_tweets_train, top_words)

In [51]:
len(dict_feat_vector)
print(dict_feat_vector[0])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [25]:
len(dict_feat_vector)

38153

El resultat serà un diccionari tipus (no necessàriament amb aquest valors):

```python
{
    0: np.array([0, 1, 1, 0, ...]),
    1: np.array([0, 1, 1, 1, ...]),
    ...
}
```

Com podem observar, hi ha un vector de característiques per cadascun dels tweets en entrenament. El que esperem és que aquest vector ens estigui donant informació del que posa a cada tweet.

In [22]:
df_tweets_train.shape

(38153, 2)

In [ ]:
# aplicar l aformula tweet representado con el vector
# cad una de las x, es independeinte las otras, seria multiplicar da cuna de lsa prob marginles

### El classificador Naïve Bayes

Un cop tenim una representació necessitem un procés d'aprenentatge que ens permeti passar de la descripció a una categoria. 
En aquest lliurament farem servir el classificador Naïve Bayes. 
Aquest classificador forma part de la família de classificadors probabilístics. 
La sortida d'un classificador probabilístic és un valor de probabilitat donat un exemple per cadascuna de les categories. 
La decisió final correspon a la categoria amb més probabilitat. 


Els classificadors probabilistics Bayesians es basen en el teorema de Bayes per realitzar els càlculs per trobar la probabilitat condicionada: 
$$ p(x,y) = p(x|y)p(y) = p(y|x)p(x)$$
d'on podem extreure que: 
$$ p(y|x) = \frac{p(x|y)p(y)}{p(x)}$$


En molts casos $p(y)$ i $p(x)$ són desconeguts i es consideren equiprobables. 
Per tant, la decisió es simplifica a:
$$ p(y|x) = p(y) · p(x|y)$$


Les deduccions fins a aquest punt són vàlides per la majoria de classificadors Bayesians. 
Naïve Bayes es distingeix de la resta perquè imposa una condició encara més restrictiva. 
Considerem $x=(x_1, \cdots, x_n)$ un conjunt d'$N$ variables aleatòries. 
Naïve Bayes assumeix que totes elles són independents entre elles i per tant podem escriure:
$$p(x_1,x_2,...,x_N | y) = p(x_1|y)p(x_2|y)...p(x_N|y)$$


Podem interpretar l'anterior equació de la següent forma: La probabilitat de que el tweet descrit pel vector de característiques (0,1,0,1,1,1) sigui de la classe "gender" és proporcional al producte de la probabilitat que la primera paraula del vector no aparegui en els tweets sobre "gender" per la probabilitat que la segona paraula sí que hi aparegui, etc.


**Estimant les probabilitats marginals condicionades**

L'últim pas que ens queda és trobar el valor de les probabilitats condicionades. 
Farem servir la representació de $0$'s i $1$'s indicant que la paraula no apareix (0) o sí apareix (1) a al tweet. 
Per trobar el valor de la probabilitat condicionada farem servir una aproximació freqüentista a la probabilitat. 
Això vol dir que calcularem la freqüència d'aparició de cada paraula per a cada categoria. 
Aquest càlcul es fa dividint el nombre de tweets de la categoria en que apareix la paraula pel nombre total de tweets d'aquella categoria. 

En gneral:
$$p(x = \text{"school"} | y = C)= \frac{A}{B} $$
on A és el número de tweets de la categoria C on hi apareix la paraula 'school' i B és el número total de tweets de la categoria C.


### Punts dèbils:

**El problema de la probabilitat 0**

Si us hi fixeu bé, la probabilitatpot ser 0 !!  Això vol dir, que si en el tweet no hi apareix una paraula no pot ser classificada com cap tipus de cyber bullying.

No sembla raonable que s'assigni o no en aquesta categoria segons si en el tweet hi apareix o no una única paraula. 
Per tant, el que s'acostuma a fer és donar una baixa probabilitat en comptes de zero. 
Una de les possibles solucions es fer servir la correcció de Laplace. 
Seguint l'exemple anterior la correcció de Laplace és
$$p(x= \text{"school"} | y = 'C' ) = \frac{A+1}{B+M}$$ 
on M és el nombre de categories

**El problema del "underflow"**

La funció que hem de calcular en el Naive Bayes és un producte. 
El nombre de caractéristiques del vector és el nombre de termes del producte. 
Aquests nombres són iguals o menors a 1, si els multipliquem tots entre ells el resultat serà massa petit per a representar-lo en un nombre de punt flotant i el càlcul acabarà sent reduït a zero. 
Per solucionar aquest problema en comptes d'operar fent multiplicacions, se sol passar a l'escala logarítmica i allà operar fent servir sumes en comptes de multiplicacions.

#### **EXERCICI 7:** 

Implementeu la funció d'aprenentatge del classificador Naïve Bayes (funció **naive_bayes_learn()**). La funció ha de mostrar per pantalla el resultat obtingut 
L'**error d'entrenament** es troba calculant el percentatge d'errors que s'obtenen quan es fa el testeig amb les mateixes dades utilizades per fer entrenament (aprenentatge). Aquest error es un valor molt optimista de com funcionarà el clasificador i mai s'ha de prendre com a mesura per comparar clasificadors. 

1) Programeu la funció **naive_bayes_learn** per a que estimi les probabilitats marginals condicionades.
2) Programeu la funció **naive_bayes** que implementa el classificador. Noteu que aquesta funció está guiada i només haureu d'emplenar els espais on hem posat tres punts suspensius "#···".

In [23]:
def naive_bayes_learn(df, feats):
    """
    Funció que estima les probabilitats marginals condicionades.
    
    :params df: DataFrame amb els tweets i la informació associada
    :params feats: vector de característiques de cada tweet
    :return : probabilitats marginals condicionades
    """
    
    # YOUR CODE HERE
    
    return probs

In [24]:
import sys
from IPython import embed
def naive_bayes(df_train, feat_train, feat_test=None, df_test=None):
    """
    Funció que implementa el clasificador Naive_Bayes.
    
    Si df_test no és None, ha de calcular l'encert sobre les dades de test. És a dir,
    després de classificar feat_test ha de comparar la classificació amb la classe
    real i dir (print) quin percentatge d'encert ha obtingut.
    
    :param df_train: DataFrame amb els tweets que s'utilitzaran per l'entrenament
    :param feat_train: Diccionari amb els vectors de caracteristiques de cada tweet de l'entrenament
    :param feat_test: Diccionari amb els vectors de caracteristiques de cada tweet de test
    :param df_test: DataFrame amb els tweets que s'utilitzaran pel test
    
    :return : Una serie on l'index correspon amb els indexos de df_test i els valors són la
        classificació retornada per Naive Bayes
    """
    probs = naive_bayes_learn(df_train, feat_train)
    p_of_cat = count_words_categories(df_train)
    p_total = len(p_of_cat.keys())
    
    def eachFeats(row):
        id, feat = row
        p_max = float('-inf')
        p_cat = 0

        for category in probs:
            # Speed up by using numpy
            # inv is the inverse of features, 0 where 1 and 1 where 0
            # ...
            
            # Probs * feats is the probability of being there, while
            # inv - inv * feat = 1 - (0, 1, 0... inverses) * probs, probability of not being there
            # ...
            
            # Sum of logs [vs] underflow caused by mul of probs
            # ...

            # Take the max, do it now to avoid extra-loops
            # ...
                
        return id, p_cat
    
    data = map(eachFeats, feat_test.items())
    data = pd.Series(dict(data))
    correct = data == df_test['cyberbullying_type']
    print("Accuracy: {}".format(correct.sum() / correct.size))
    
    return correct.sum() / correct.size

In [25]:
N = 20 # Aquest parametre el podem canviar i fer proves per avaluar quin és el millor valor. 
words_topics = count_words_categories(df_tweets_train)
top_words = topNwords(df_tweets_train, words_topics, N, skip_top)

feat_train = create_features(df_tweets_train, top_words)
feat_test = create_features(df_tweets_test, top_words)

In [26]:
accuracy = naive_bayes(df_tweets_train, feat_train, feat_test, df_tweets_test)

Accuracy: 0.6931544187021701


Haurieu d'obtenir una accuracy del 69-70%. Si heu arribat a això ja està bé! 

En canvi, per aquells que vulguin tenir alguns **PUNTS EXTRA**, us retem a aconseguir una accuracy més alta. A veure què podeu fer!!!



**IMPLEMENTACIÓ DE MILLORA**:

In [ ]:
# 